<a href="https://colab.research.google.com/github/OussamaHaff/machine-learning-upskilling/blob/main/02-llms-from-scratch/04-token-embeddings-with-positional-encoding/token_embeddings_with_positisional_encoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic token IDs to embedding vectors

## Exploration

In [21]:
import torch

input_token_ids = torch.tensor([2, 3, 5, 1])

vocab_size = 6
output_embeddings_dim_size = 3

# Instantiate an embedding layer (with random seed 123 for reproducibility)
torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(num_embeddings=vocab_size,
    embedding_dim=output_embeddings_dim_size)
print("Wights \n", embedding_layer.weight)
print("Shape\n", embedding_layer.weight.shape)

Wights 
 Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)
Shape
 torch.Size([6, 3])


In [18]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


In [19]:
print(embedding_layer(input_token_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


## Observations

- Initial Embedding weights are random.
- The Embedding layer is essentially a lookup function to get wights of of token IDs.

## Mini positional embedding

In [75]:
max_length = 4
pos_embedding_layer = torch.nn.Embedding(
    num_embeddings=max_length, embedding_dim=output_embeddings_dim_size)

positions = torch.arange(max_length)

pos_embeddings = pos_embedding_layer(positions)

mini_embedding = embedding_layer(input_token_ids)

mini_input_embeddings = mini_embedding + pos_embeddings

print("Embeddings")
print(mini_embedding)
print("Positions")
print(positions)
print("Positional Embeddings")
print(pos_embeddings)
print("Target Embeddings")
print(mini_input_embeddings)

Embeddings
tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)
Positions
tensor([0, 1, 2, 3])
Positional Embeddings
tensor([[ 1.1621,  0.0253,  0.5817],
        [-0.5675, -0.8296, -0.9905],
        [ 0.0182,  0.6924, -1.5154],
        [ 0.5716,  0.5938, -0.9716]], grad_fn=<EmbeddingBackward0>)
Target Embeddings
tensor([[ 2.4374, -0.1756,  0.4211],
        [-0.9690,  0.1370, -2.1387],
        [-2.8218, -0.0924, -2.9250],
        [ 1.4894,  2.1748,  0.3294]], grad_fn=<AddBackward0>)


# Embedding with positional encoding

## From previous step

In [30]:
# Google Collab
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.4 MB/s eta 0:00:00


In [31]:
import torch
import tiktoken
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
  """Contructs a PyTorch Dataset with """
  def __init__(self, text, tokeniser, sliding_window_max_length, stride):
    self.input_ids = []
    self.target_ids = []

    tokenised_text = tokeniser.encode(text)

    for i in range(0, len(tokenised_text) - sliding_window_max_length, stride):
      input_chunk = tokenised_text[i: i + sliding_window_max_length]
      target_chunk = tokenised_text[i + 1 : i + sliding_window_max_length + 1]
      self.input_ids.append(torch.tensor(input_chunk))
      self.target_ids.append(torch.tensor(target_chunk))

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, id):
    return self.input_ids[id], self.target_ids[id]


def create_dataloader_v1(text, batch_size=4, sliding_window_max_length=256,
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):
  tokeniser = tiktoken.get_encoding("gpt2")
  dataset = GPTDatasetV1(text, tokeniser, sliding_window_max_length, stride)
  dataloader = DataLoader(
      dataset,
      batch_size=batch_size,
      shuffle=shuffle,
      drop_last=drop_last,
      num_workers=num_workers
  )
  return dataloader


with open("data/the-verdict.txt", "r", encoding="utf-8") as dataset_file:
  raw_text = dataset_file.read()

In [32]:
vocab_size = 50257
output_dimension = 256

token_embedding_layer = torch.nn.Embedding(num_embeddings=vocab_size, embedding_dim=output_dimension)
print(token_embedding_layer.weight.shape)

torch.Size([50257, 256])


In [41]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, sliding_window_max_length=max_length,
    stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

print("Token IDs:\n", inputs)
print("Input shape:\n", inputs.shape)


Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
Input shape:
 torch.Size([8, 4])


In [52]:
tokens_embeddings = token_embedding_layer(inputs)

print(tokens_embeddings.shape)

torch.Size([8, 4, 256])


In [53]:
context_length = max_length
positional_embedding_layer = torch.nn.Embedding(
    num_embeddings=context_length, embedding_dim=output_dimension)
positional_embeddings = positional_embedding_layer(torch.arange(context_length))

print(positional_embeddings.shape)


torch.Size([4, 256])


In [58]:
input_embeddings = tokens_embedding + positional_embeddings
print(input_embeddings)

tensor([[[ 1.1555, -0.3882, -0.6872,  ...,  0.3243,  3.4303, -0.8312],
         [-1.6216, -0.2573, -1.7572,  ..., -0.4452, -2.4622,  0.1155],
         [-0.9715,  1.4540,  0.4088,  ...,  0.9573,  2.0514, -0.2320],
         [-0.8488,  1.1452,  0.1831,  ...,  0.5796,  2.0372, -1.4826]],

        [[ 2.2112,  1.1132, -0.3223,  ...,  0.4431,  2.2109, -0.1119],
         [-1.0261, -1.0020, -0.6953,  ..., -1.5052, -1.7355,  0.9296],
         [-0.5452,  0.0628,  0.2625,  ...,  0.8108, -0.0856, -0.0732],
         [-0.8441,  2.7050,  0.8911,  ..., -0.2137,  0.2933, -0.0098]],

        [[ 1.6476, -0.5286, -1.2389,  ...,  1.1974,  0.8194, -2.0030],
         [-0.5285, -0.5203, -0.9484,  ..., -0.4651, -3.3541,  0.7916],
         [ 1.2956,  1.6018,  0.3258,  ..., -0.7885,  0.7958, -1.1924],
         [-0.2042,  0.5778,  0.8281,  ...,  1.5140, -0.1738, -0.4808]],

        ...,

        [[ 2.0647,  1.1278,  0.1049,  ...,  1.1730,  0.0323,  1.2470],
         [-2.0486, -0.8185, -2.0130,  ..., -1.6087, -1.77